In [2]:
import time

from sklearn.datasets import fetch_california_housing
from sklearn.linear_model import LinearRegression, SGDRegressor, Ridge, LogisticRegression, Lasso
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, classification_report, roc_auc_score
import joblib
import pandas as pd
import numpy as np

In [3]:
"""
线性回归直接预测房子价格
:return: None
"""
# 获取数据
lb = fetch_california_housing(data_home='../MachineData/data')

print("获取特征值")
print(lb.data.shape)
print(lb.data[0])
# print("目标值")
# print(lb.target)
# print(lb.DESCR)
print(lb.feature_names)
print('-' * 50)
# 分割数据集到训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(lb.data, lb.target, test_size=0.25, random_state=1)
#
print(x_train.shape)

获取特征值
(20640, 8)
[   8.3252       41.            6.98412698    1.02380952  322.
    2.55555556   37.88       -122.23      ]
['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
--------------------------------------------------
(15480, 8)


In [4]:
y_train
# x_train

array([1.034, 1.114, 2.164, ..., 1.048, 1.407, 1.26 ])

In [5]:
# # 进行标准化处理(?) 目标值处理？
# # 特征值和目标值是都必须进行标准化处理, 实例化两个标准化API
std_x = StandardScaler()
#
std_x.fit(x_train) #计算均值和方差
# # 对训练集和测试集的特征值进行标准化
x_train = std_x.transform(x_train)
# x_train = std_x.fit_transform(x_train)
x_test = std_x.transform(x_test)

# 目标值进行了标准化,也可以不进行标准化
std_y = StandardScaler()
#
temp = y_train.reshape(-1, 1) #-1代表把剩余的元素都堆到哪一维
temp

array([[1.034],
       [1.114],
       [2.164],
       ...,
       [1.048],
       [1.407],
       [1.26 ]])

In [6]:
#这个单元格不能运行多次
#标签进行标准化
# 目标值是一维的，这里需要传进去2维的
y_train = std_y.fit_transform(y_train.reshape(-1, 1))
print(y_train.shape)
y_test = std_y.transform(y_test.reshape(-1, 1))
print(y_test.shape)

(15480, 1)
(5160, 1)


In [ ]:
# test1=np.array([1,2,3])
# test1.reshape(-1,1)

In [11]:
# # estimator预测
# # # 正规方程求解方式预测结果，正规方程进行线性回归
lr = LinearRegression()
# fit是耗时的
start=time.time()
lr.fit(x_train, y_train)
end=time.time()
print('耗时',end-start) #训练耗费时间
#回归系数可以看特征与目标之间的相关性,每个特征都有一个回归系数
print('回归系数', lr.coef_)
#
y_predict = lr.predict(x_test)
# 预测测试集的房子价格，通过inverse得到真正的房子价格
y_lr_predict = std_y.inverse_transform(y_predict)
# 保存训练好的模型，模型中保存的是w的值，也保存了模型结构
#保存模型放在fit之后即可
joblib.dump(lr, "./12code-save/test.pkl")#放在tmp文件夹下
print("正规方程  测试集里面每个房子的预测价格：", y_lr_predict[0:10])
#下面是求测试集的损失，也就是均方误差，求平方和再求平均
print("正规方程的均方误差：", mean_squared_error(y_test, y_predict))

耗时 0.003002166748046875
回归系数 [[ 0.71942632  0.10518431 -0.23147194  0.26802332 -0.00448136 -0.03495117
  -0.7849086  -0.76307353]]
正规方程  测试集里面每个房子的预测价格： [[2.12391852]
 [0.93825754]
 [2.7088455 ]
 [1.70873764]
 [2.82954754]
 [3.50376456]
 [3.0147162 ]
 [1.62781292]
 [1.74317518]
 [2.01897806]]
正规方程的均方误差： 0.40082431136214186


# 2 加载保存的模型

In [13]:
#上一个代码，没有保存模型，让我给注释掉了
model = joblib.load("./12code-save/test.pkl")
# # 因为目标值进行了标准化，一定要把预测后的值逆向转换回来
y_predict = model.predict(x_test)

#
print("保存的模型预测的结果：", y_predict)
print("正规方程的均方误差：", mean_squared_error(y_test, y_predict))

print("正规方程inverse后的均方误差：", mean_squared_error(std_y.inverse_transform(y_test),
                                               std_y.inverse_transform(y_predict)))

保存的模型预测的结果： [[ 0.039975  ]
 [-0.9856667 ]
 [ 0.54595901]
 ...
 [-0.72237246]
 [ 0.57093569]
 [-0.27655325]]
正规方程的均方误差： 0.40082431136214186
正规方程inverse后的均方误差： 0.5356532845422556


In [ ]:
#理解均方误差
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]
mean_squared_error(y_true, y_pred)

In [24]:
#人工求均方误差
(np.square(3 - 2.5) + np.square(0.5) + 1) / 4

0.375

In [25]:
# 梯度下降去进行房价预测,数据量大要用这个
# learning_rate设置为不同的字符串，代表学习率变化的算法不一样
# eta0 是learning_rate的初始值
# learning_rate='optimal',alpha是正则化力度，但是会影响学习率的值，由alpha来算学习率
# penalty代表正则化，分为l1和l2
#每次训练得到损失不一样，是因为w每次随机的初始值不一样
sgd = SGDRegressor(eta0=0.01, penalty='l1',learning_rate='invscaling')
# # 训练
sgd.fit(x_train, y_train)
#回归系数可以看特征与目标之间的相关性,每个特征都有一个回归系数
print('梯度下降的回归系数', sgd.coef_)
#
# 预测测试集的房子价格
y_sgd_predict = std_y.inverse_transform(sgd.predict(x_test).reshape(-1, 1))
y_predict = sgd.predict(x_test)
# print("梯度下降测试集里面每个房子的预测价格：", y_sgd_predict)
print("梯度下降的均方误差：", mean_squared_error(y_test, y_predict))
print("梯度下降的原始房价量纲均方误差：", mean_squared_error(std_y.inverse_transform(y_test), y_sgd_predict))




梯度下降的回归系数 [ 0.72683307  0.08361568 -0.2929425   0.15964194 -0.00588591 -0.05732222
 -0.7614855  -0.74102704]
梯度下降的均方误差： 0.42066585490750663
梯度下降的原始房价量纲均方误差： 0.5621691112253838


D:\PY\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
